<a href="https://colab.research.google.com/github/cincinnatilibrary/collection-analysis/blob/master/misc/chpl_bulk_connected_cards.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img src="https://raw.githubusercontent.com/cincinnatilibrary/collection-analysis/master/misc/CHPL_Brandmark_Primary.png" alt="CHPL" title="CHPL" width="250"/>

# REST API: Bulk ConnectEd / Student Card Creation


**Additional instructions here:** 

* Populate the following template with data that will be used to create patron records in bulk for ConnectED accounts

  https://github.com/cincinnatilibrary/collection-analysis/raw/master/misc/ConnectEd-TEMPLATE.xlsx

  _Note_: the column, "Alt ID (optional)" is in fact optional, and can be left blank when not needed.

---
---

## 1. Enter REST-API Credentials

In order to be authorized to use the REST API, you must enter valid credientials.

 ↴ **Click on the play button below**, then when prompted, enter in the **`auth_string`** as your password:

In [ ]:
#@title
# <--- Click the play button to the left to get started
import time

# this is the helper script that contains some needed functions
!wget --quiet https://raw.githubusercontent.com/cincinnatilibrary/collection-analysis/8352bf7d9debf027e760e426c6d54a1df3aee77c/misc/chpl_helper.py --output-document=chpl_helper.py
!pip install -U rich > /dev/null
time.sleep(1)

# import the helper functions we just downloaded ...
import chpl_helper
from getpass import getpass, getuser
from google.colab import files
import pandas as pd
import requests
import json
from io import StringIO
from rich.progress import Progress, SpinnerColumn, TextColumn, track
from rich import print
import logging
from math import isnan

# create log file
ts = pd.Timestamp('now').isoformat()
logging.basicConfig(
    filename="bulk_connected.txt", 
    format="%(asctime)s\t%(levelname)s\t%(message)s", 
    level=logging.DEBUG
)
logging.info('script started: {}'.format(ts))

# point this to the base URL for the Sierra REST API
base_url = 'https://classic.cincinnatilibrary.org:443/iii/sierra-api/v6/'

# client_key = getpass('Enter the client KEY value: ')
# client_secret = getpass('Enter the client SECRET value: ')
auth_string = getpass('Enter the auth_string value: ')

# test our credentials

try:
  headers = chpl_helper.set_access_headers(base_url, auth_string=auth_string)
  r = requests.get(base_url + 'info/token', headers=headers, verify=True)
  print('access token expires in: {} seconds'.format(r.json()['expiresIn']))
  logging.info('keyId: {}'.format(r.json()['keyId']))
  logging.info('expiresIn: {}'.format(r.json()['expiresIn']))
except:
  logging.error('requst info/token status:{} \tcontent:'.format(r.status_code, r.content))


## 2. Upload Input File

This file will be processed to placed the holds.

 ↴ **Click on the play button below**, then when prompted, upload the input file:

In [ ]:
# convert the input file to a Dataframe ... 
#@title
input_file = files.upload()

# convert the input file to a Dataframe ... 
df = pd.read_excel( 
    [key for key in input_file.keys()][-1], # this is the last file uploaded
    usecols=range(18), # use all columns
    converters=dict.fromkeys(range(18), str), # treat all columns as strings 
)

# column 'Birth Date' should in the format %m%d%Y e.g. `01251977` 
df[df.columns[11]] = pd.to_datetime(
    df[df.columns[11]], 
    format='%m%d%Y'
)

# print(df.head(5))
# print('...')
msg = f"total rows in input: {df.shape[0]}"
print(msg)
logging.info([key for key in input_file.keys()][-1])
logging.info(msg)
del(msg)
df.shape[0] 
print(
    'first 3 rows, and last 3 rows\n---\n',
    pd.concat(
      [df[:3], df[-3:]], 
      sort=False
    )
)

headers = chpl_helper.set_access_headers(base_url, auth_string=auth_string)

# try:
#   text_column = TextColumn("Searching")
#   spinner_column = SpinnerColumn(finished_text='Complete!')
#   with Progress(text_column, spinner_column, expand=False) as progress:
#   #Progress(transient=True) as progress:
#     task = progress.add_task("Searching ...")

#     r = requests.post(
#       base_url + 'items/query?offset=0&limit={}'.format(len(operands)), 
#       headers=headers,
#       data=json.dumps(item_query),
#       verify=True
#     )
#     progress.update(task, completed=1)

#   # print("response json: ", r.json())
#   print('...')
#   msg = 'total rows of items returned from search: {}'.format(r.json()['total'])
#   logging.info(msg)
#   print(msg)

# except:
#   logging.error('items/query failed')
#   print('error?')

## 4. Create the patron records ...:

 ↴ **Click on the play button below**, for the patron records to create from the data above
 :

In [ ]:
#@title
from math import isnan
class PatronNew:
    """
    defines a CUSTOMIZED patron record object for use with the API
    ... pulling in the variable data in as arguments in the __init__ function
    ... and setting constant values in the __init__ function

    """

    """
    TODO: more docs here about the object it creates
    """
    def __init__(self,
                 last_name,
                 first_name,
                 barcode,
                 student_id,
                 school_district,
                 pin,
                 school,
                 birth_date,
                 phone_number,
                 home_legal_address,
                 home_legal_address_city,
                 home_legal_address_state,
                 home_legal_address_zip,
                 notice_pref=None,
                 email_address=None,
                 home_mailing_address=None,
                 home_mailing_address_city=None,
                 home_mailing_address_state=None,
                 home_mailing_address_zip=None,                 
                 home_library_code=None,
                 patron_agency=None,
                 alt_id=None,
    ):
        
        # patron data will be a dictionary that will be converted to json and passed to the API
        
        # define some fields we can use for easy access
        self.alt_id = alt_id
        
        self.birth_date = birth_date

        # expiration date is 3 years from when these records are created
        # expiration_date = '2023-09-03'
        expiration_date = (pd.Timestamp.now() + pd.DateOffset(years=3)).strftime('%Y-%m-%d')
        
        # uses the pd.Timedelta object to determine the number of years old
        self.years_old = int( (pd.Timestamp.now() - birth_date).days / 365)
        if (self.years_old >= 18):
            self.patron_type = 3
        elif (self.years_old >= 13):
            self.patron_type = 2
        else:
            self.patron_type = 1
            
        # normalize the school name 
        self.school = school.lower().title()

        # address data:
        addresses_line1 = home_legal_address
        addresses_line2 = home_legal_address_city + ', ' + home_legal_address_state + ' ' + home_legal_address_zip
        
        # this is the data object that matches the patronPatch object
        self.patron_data = {
            
            # EXAMPLE of how to formatat dates coming in as '%m-%d-%Y', and converted to %Y-%m-%d (isoformat)
            #"expirationDate": str(datetime.strptime(str(expiration_date), "%m-%d-%Y").date().isoformat()),
            'expirationDate': expiration_date,
            # 'birthDate': '2015-09-15',
            'birthDate': birth_date.strftime('%Y-%m-%d'),
            'patronType': self.patron_type,
            'blockInfo': {'code': '-'},
            'phones': [{'number': str(phone_number), 'type': 't'}],
            #
            # add this after creating the initial object
            #'emails': [email_address],
            'pMessage': '-',
            'fixedFields': {
                '44': {'label': 'E-Lib Update? (P1)', 'value': 'n'},
                '45': {'label': 'Friends? (P2)', 'value': 'n'},
                '46': {'label': 'Foundation? (P3)', 'value': '1'},
                # don't set Birth Date as a fixed field ... even though there's the option?! ... yes.
                # '51': {'label': 'Birth Date', 'value': '2015-09-15'},

                #
                # TODO:
                # we either need to figure out if this needs a label or 
                #
                # '86': {'label': 'Agency', 'value': '38', 'display': 'Symmes Township'},
                '86': {'label': 'Agency', 'value': str(patron_agency)},
                # TODO: find out if this is needed
                # NOT SURE IF THIS IS NEEDED
                #'126': {'label': 'County (P4)', 'value': '202'},
                '158': {'label': 'Patron Agency', 'value': str(patron_agency)},
                # NOTE, this gets changed below (if set), but defaults to 'email'
                '268': {'label': 'Notice Preference', 'value': 'z'}
            },
            
            "names": [
                str(last_name) + ", " + str(first_name)
            ],
            "barcodes": [
                str(barcode)
            ],
            "homeLibraryCode": str(home_library_code),
            "varFields": [
                # again, i'm not sure why there are so many ways you could set the birthdate ... 
                # it just appears to require being set in a varfield as well as in the "primary"
                # field tag d = birthDate (MMDDYYYY)
                {
                    'fieldTag': 'd',
                    'content': birth_date.strftime('%m%d%Y')
                    # 'content': '09152015'
                },
                {
                    "fieldTag": 'x',
                    "content": 'ConnectED'
                },
                {
                    "fieldTag": "l",
                    "content": self.school
                },
                
                
            ],
            "addresses": [
                {
                    "lines": [
                        str(addresses_line1),
                        str(addresses_line2)
                    ],
                    "type": "a"
                },
                # possibly append to addresses below...
            ],

            "pin": str(pin),
        }

        # email maybe left blank ...
        if isnan(email_address) is False:
          self.patron_data['emails'] = [email_address]

        if notice_pref is not None:
          self.patron_data['fixedFields']['268'] = {'label': 'Notice Preference', 'value': str(notice_pref)}

        if alt_id is not None:
          self.patron_data['varFields'].append(
              {
                  "fieldTag": "v",
                  "content": str(alt_id)
              }              
          )


import re

# # run the test for the first line...
# test_patron_data = df.iloc[0]

# test_patron = PatronNew(
#     home_library_code=test_patron_data[0],
#     patron_agency=test_patron_data[1],
#     notice_pref=test_patron_data[2],
#     last_name=test_patron_data[3],
#     first_name=test_patron_data[4],
#     barcode=test_patron_data[5],
#     pin=test_patron_data[6],
#     student_id=test_patron_data[7],
#     school_district=test_patron_data[8],
#     school=test_patron_data[9],
#     birth_date=test_patron_data[10],
#     email_address=test_patron_data[11],
#     phone_number=test_patron_data[12],
#     home_legal_address=test_patron_data[13],
#     home_legal_address_city=test_patron_data[14],
#     home_legal_address_state=test_patron_data[15],
#     home_legal_address_zip=test_patron_data[16]
# )

# json.dumps(test_patron.patron_data)

# match the patron record number at the end of the string
patron_num_re = re.compile(r"^.*\/([0-9].*)$")

for i, row in df.iterrows():

  patron_obj = PatronNew(
    home_library_code=row[0],
    patron_agency=row[1],
    notice_pref=row[2],
    last_name=row[3],
    first_name=row[4],
    barcode=row[5],
    pin=row[6],
    student_id=row[7], 
    alt_id=row[8], # this is optional
    school_district=row[9],
    school=row[10],
    birth_date=row[11],
    email_address=row[12],
    phone_number=row[13],
    home_legal_address=row[14],
    home_legal_address_city=row[15],
    home_legal_address_state=row[16],
    home_legal_address_zip=row[17]
  )

  # print(json.dumps(patron_obj.patron_data))

  try:
    r_patron = requests.post(
        url=base_url + 'patrons/',
        headers=headers,
        data=json.dumps(patron_obj.patron_data),
        verify=True
    )

    patron_num = int(patron_num_re.match(r_patron.json()['link'])[1])
    logging.info(patron_num)
    print(i, ',p', patron_num, 'a', sep='')
    
  except:
    logging.error(f"{i} POST request not successful")
    print('-1')          

## 4a. Download log file:

 ↩ **Click on the "Files" button (image.png) to the left, right the file `bulk_connected.txt` and then click `Download`**